In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'mps'

In [24]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
if device == "cuda":
	model = torch.optimize(model)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [25]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [34]:
def train_model(model, loss_fn, optimizer):
	epochs = 10
	for t in range(epochs):
		print(f"Epoch {t+1}\n-------------------------------")
		train(train_dataloader, model, loss_fn, optimizer)
		test(test_dataloader, model, loss_fn)
	print("Done!")

In [26]:
train_model(model, loss_fn, optimizer)

Epoch 1
-------------------------------
loss: 2.301783  [   64/60000]
loss: 2.290540  [ 6464/60000]
loss: 2.269717  [12864/60000]
loss: 2.265140  [19264/60000]
loss: 2.238884  [25664/60000]
loss: 2.212539  [32064/60000]
loss: 2.219168  [38464/60000]
loss: 2.186257  [44864/60000]
loss: 2.182139  [51264/60000]
loss: 2.140126  [57664/60000]
Test Error: 
 Accuracy: 45.6%, Avg loss: 2.140773 

Epoch 2
-------------------------------
loss: 2.152323  [   64/60000]
loss: 2.143236  [ 6464/60000]
loss: 2.081924  [12864/60000]
loss: 2.101945  [19264/60000]
loss: 2.033799  [25664/60000]
loss: 1.976586  [32064/60000]
loss: 2.003998  [38464/60000]
loss: 1.920673  [44864/60000]
loss: 1.930494  [51264/60000]
loss: 1.846056  [57664/60000]
Test Error: 
 Accuracy: 57.6%, Avg loss: 1.848725 

Epoch 3
-------------------------------
loss: 1.884627  [   64/60000]
loss: 1.853767  [ 6464/60000]
loss: 1.733704  [12864/60000]
loss: 1.780946  [19264/60000]
loss: 1.651876  [25664/60000]
loss: 1.617443  [32064/600

In [53]:
rand = torch.rand(1, 28, 28).to(device)
rand.shape
pred = model(rand)

In [55]:
pred.shape

torch.Size([1, 10])

In [57]:
path = "NN.pt"
model.eval()

# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 28, 28).to(device)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save(path)

print(f"Saved PyTorch Model script to {path}")

Saved PyTorch Model script to NN.pt


In [38]:
from torchvision.models import resnet18

class MnistResNet(nn.Module):
  def __init__(self, in_channels=1):
    super(MnistResNet, self).__init__()

    # Load a pretrained resnet model from torchvision.models in Pytorch
    self.model = resnet18(weights=False)

    # Change the input layer to take Grayscale image, instead of RGB images.
    # Hence in_channels is set as 1 or 3 respectively
    # original definition of the first layer on the ResNet class
    # self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
    self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

    # Change the output layer to output 10 classes instead of 1000 classes
    num_ftrs = self.model.fc.in_features
    self.model.fc = nn.Linear(num_ftrs, 10)

  def forward(self, x):
    return self.model(x)


resnet = MnistResNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet.parameters(), lr=1e-3)
resnet

MnistResNet(
  (model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [39]:
train_model(resnet, loss_fn, optimizer)

Epoch 1
-------------------------------
loss: 2.563664  [   64/60000]
loss: 1.207950  [ 6464/60000]
loss: 0.595497  [12864/60000]
loss: 0.824724  [19264/60000]
loss: 0.736061  [25664/60000]
loss: 0.650119  [32064/60000]
loss: 0.565210  [38464/60000]
loss: 0.682832  [44864/60000]
loss: 0.587535  [51264/60000]
loss: 0.559341  [57664/60000]
Test Error: 
 Accuracy: 81.4%, Avg loss: 0.516020 

Epoch 2
-------------------------------
loss: 0.466861  [   64/60000]
loss: 0.597368  [ 6464/60000]
loss: 0.252069  [12864/60000]
loss: 0.534733  [19264/60000]
loss: 0.477952  [25664/60000]
loss: 0.498202  [32064/60000]
loss: 0.412848  [38464/60000]
loss: 0.543477  [44864/60000]
loss: 0.451041  [51264/60000]
loss: 0.476975  [57664/60000]
Test Error: 
 Accuracy: 83.9%, Avg loss: 0.447082 

Epoch 3
-------------------------------
loss: 0.339519  [   64/60000]
loss: 0.480570  [ 6464/60000]
loss: 0.187824  [12864/60000]
loss: 0.437306  [19264/60000]
loss: 0.379235  [25664/60000]
loss: 0.419831  [32064/600

In [59]:
path = "resnet.pt"
resnet.eval()

# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 28, 28).to(device)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(resnet, example.unsqueeze(0))
traced_script_module.save(path)

print(f"Saved PyTorch Model script to {path}")

Saved PyTorch Model script to resnet.pt
